In [64]:
# Input data.
# Since the classes are unequal, we don't risk random sampling from full set.
# Instead, keep them separate until we make the train/test split.
infile_pos='score-positive.4mer.features.csv'
infile_neg='score-negative.4mer.features.csv'
infile_zero='score-zero.4mer.features.csv'
import numpy as np
import pandas as pd
raw_pos = pd.read_csv(infile_pos,header=0)
raw_neg = pd.read_csv(infile_neg,header=0)
raw_zero = pd.read_csv(infile_zero,header=0)
raw_pos.head()

,label,seqname,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,1,ENSG00000261061,31,5,2,13,6,5,0,5,...,0,2,5,5,2,4,2,3,4,7
1,1,ENSG00000254837,87,19,28,42,31,9,1,15,...,1,15,13,13,6,14,17,11,10,30
2,1,ENSG00000282851,105,29,43,51,48,23,2,23,...,3,21,21,20,14,27,28,23,22,62
3,1,ENSG00000255650,108,32,51,55,55,25,3,26,...,3,24,21,21,19,30,28,28,25,83
4,1,ENSG00000163597,133,47,74,80,65,39,8,42,...,6,45,37,32,37,61,66,55,51,169


In [67]:
def split_train_test(dataset,train_portion):    
    # Use pandas sample() to randomize the order (i.e. random sample without replacement).
    # Side note. Data frame can be randomized while keeping header in place.
    # This is repeatable with 42.
    middle = int(len(dataset)*train_portion)
    random = dataset.sample(frac=1,random_state=42)
    train_set = random[:middle]
    test_set = random[middle:]
    return train_set,test_set
TRAIN_PORTION=0.8

train_pos , test_pos = split_train_test(raw_pos,TRAIN_PORTION)
train_neg , test_neg = split_train_test(raw_neg,TRAIN_PORTION)
train_zero , test_zero = split_train_test(raw_zero,TRAIN_PORTION)

raw_pos.shape, train_pos.shape, test_pos.shape

((170, 258), (136, 258), (34, 258))

In [81]:
# The Geron book recommends this (or StrattifiedShuffleSplit) 
# but we could not make it work. 

#from sklearn.model_selection import ShuffleSplit
#splitter=ShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
#for train_index,test_index in splitter.split(rawpos):
#    train_pos = rawpos[train_index]
#    test_pos = rawpos[test_index]

In [93]:
# Combine and shuffle.
train_sorted = pd.concat([train_pos, train_neg, train_zero],axis=0)
train_set = train_sorted.sample(frac=1,random_state=17)

test_sorted = pd.concat([test_pos, test_neg, test_zero],axis=0)
test_set = test_sorted.sample(frac=1,random_state=17)

train_set.shape, test_set.shape

((921, 258), (231, 258))

In [105]:
Y_train = train_set['label']
X_train_ID = train_set['seqname']
X_train = train_set.drop(['label','seqname'],axis=1)
Y_train.unique()
X_train.shape
X_train_ID.head()
X_train.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
165,4148,1804,2268,2814,1872,1279,365,1532,2230,1468,...,291,2472,1658,1396,1723,1812,2329,2368,2403,4475
61,1667,731,923,1120,739,508,163,593,866,607,...,133,966,685,557,721,715,933,963,997,1943
505,12036,5321,6893,7671,5545,4042,1123,4564,6722,4823,...,1184,7684,4816,4459,5683,5710,6584,7357,7277,13514
195,4692,2035,2540,2860,2096,1497,431,1606,2489,1812,...,448,2771,1756,1639,2124,2084,2365,2704,2658,5104
28,732,328,399,483,332,248,69,247,377,280,...,54,476,302,246,308,296,390,441,438,882
